In [65]:
import ast
import json
import os

import numpy as np
import pandas as pd
from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
#import plotly
#plotly.offline.init_notebook_mode(connected=True)
#import plotly.offline as py
import plotly.plotly as py
import plotly.tools as tls
import string
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
from src.data import read_files
from collections import Counter
import itertools
from textblob import TextBlob
from langdetect import detect

import string
from googletrans import Translator
#from google.api_core.protobuf_helpers import get_messages
#from __future__ import absolute_import
from google.cloud import translate
from langdetect import detect
translator = Translator()
import math

import sys
reload(sys)
sys.setdefaultencoding('utf-8')

import csv

In [66]:
#hotel = pd.DataFrame.from_csv('./data/hotel_train.csv')
hotel = pd.read_csv('./data/hotel_phrase.csv')
foreign = pd.DataFrame.from_csv('./data/foreign.csv')
hotel['words_clean'] = hotel['text_clean'].apply(lambda x: str(x).split(' '))
hotel['words_clean'][foreign.index] = foreign['clean_words']
hotel['text_clean'][foreign.index] = foreign['text_clean']
hotel['phrases'][foreign.index] = foreign['phrases']
hotel['words_clean'] = hotel['words_clean'].apply(lambda x: set(str(x).split(' ')))     #get unique word shown in each review

In [120]:
def count_fre(stars, ch):
    """
    count the frequency of 'ch': could be phrases or words
    corresponding to the 'stars': could be 1, 2, 3, 4, 5
    """
    p_list = list(itertools.chain(*list(hotel.loc[hotel['stars']==stars, ch])))
    p_count = pd.DataFrame(Counter(p_list).most_common())
    p = pd.DataFrame(p_count[p_count.columns[1]])
    p.index = p_count[p_count.columns[0]]
    p.columns = [str(stars)]
    return p

def str_to_list(s):
    """
    transform a string to a list, e.g. "[a,b,c]" to [a,b,c] 
    """
    s = s.replace("u'", "")
    s = s.replace("'", "")
    l = s[1:-1].split(',')
    return l

from scipy.stats import spearmanr
def get_spearman_corr(row):
    """
    calculate the spearman's correlation coeffecient: for both linear and nonlinear relation
    """
    x = [row['1'],row['2'],row['3'],row['4'],row['5']]
    return spearmanr(x,range(1,6))[0]

def clean_word_set(x):
    """
    to delete ' , [ ] that showed after setting the unique words
    """
    s = [i.replace("'",'') for i in x]
    s = [i.replace(",",'') for i in s]
    s = [i.replace("[",'') for i in s]
    s = [i.replace("]",'') for i in s]
    return s

In [68]:
#counts of phrases
hotel['phrases'] = hotel['phrases'].apply(lambda x: str_to_list(x))
p = pd.concat([count_fre(1, 'phrases'), count_fre(2,'phrases'),count_fre(3,'phrases'),count_fre(4,'phrases'),count_fre(5,'phrases')],axis=1)
p = p.sort_values(by=['1'], ascending=False)
p = p.fillna(0)

In [123]:
#counts of words
hotel['words_clean'] = hotel['words_clean'].apply(lambda x: clean_word_set(x))
w = pd.concat([count_fre(1, 'words_clean'), count_fre(2,'words_clean'),count_fre(3,'words_clean'),count_fre(4,'words_clean'),
               count_fre(5,'words_clean')],axis=1)
w = w.sort_values(by=['1'], ascending=False)
w = w.fillna(0)

In [71]:
#######you could start from here if you wanna rerun the code########
w.to_csv('./data/hotel_words_count.csv',index=False)
p.to_csv('./data/hotel_phrase_count.csv',index=False)
#w = pd.read_csv('./data/hotel_words_count.csv')
#p = pd.read_csv('./data/hotel_phrase_count.csv')

In [76]:
p.ix[:,'1':'5'] = p.ix[:,'1':'5']/Counter(hotel['stars']).values()   #scale the importance of features under each stars level
w.ix[:,'1':'5'] = w.ix[:,'1':'5']/Counter(hotel['stars']).values()

In [77]:
p['correlation'] = p.apply(get_spearman_corr,axis=1) 
w['correlation'] = w.apply(get_spearman_corr,axis=1)

In [132]:
p[(p['correlation']<-0.9) & (p['1']>0.0015)].sort_values(by=['correlation'])

,1,2,3,4,5,correlation
credit card,0.012069,0.006353,0.002919,0.001305,0.000891,-1.000000
numerous times,0.002075,0.001485,0.000730,0.000700,0.000537,-1.000000
line hour,0.001943,0.000863,0.000324,0.000159,0.000110,-1.000000
check room,0.001928,0.001692,0.000892,0.000509,0.000439,-1.000000
debit card,0.001928,0.001139,0.000622,0.000334,0.000232,-1.000000
new keys,0.001928,0.001312,0.000757,0.000223,0.000122,-1.000000
open door,0.001913,0.001554,0.000838,0.000589,0.000244,-1.000000
poor service,0.001899,0.001692,0.000405,0.000095,0.000073,-1.000000
gon na,0.001855,0.001381,0.001081,0.000891,0.000573,-1.000000
fly airline,0.001855,0.000276,0.000081,0.000064,0.000024,-1.000000


In [92]:
p[(p['correlation']>0.85) & (p['5']>0.002)].sort_values(by=['correlation'], ascending = False)

,1,2,3,4,5,correlation
time vegas,0.002223,0.003246,0.003324,0.004121,0.004611,1.0
vegas trip,0.000662,0.000932,0.001243,0.001480,0.002062,1.0
great place,0.000368,0.000725,0.001865,0.005171,0.005453,1.0
great service,0.001678,0.002728,0.003189,0.006889,0.013187,1.0
great staff,0.000191,0.001070,0.001757,0.003436,0.004770,1.0
beautiful hotel,0.000559,0.001899,0.002567,0.003834,0.004026,1.0
super helpful,0.000191,0.000414,0.000622,0.001352,0.002269,1.0
top notch,0.000147,0.000276,0.000811,0.001543,0.003684,1.0
great way,0.000618,0.000794,0.001108,0.002386,0.004014,1.0
great food,0.000339,0.001588,0.001676,0.003818,0.005660,1.0


In [130]:
w[(w['correlation']<-0.95) & (w['1']>0.09)].sort_values(by=['correlation'])

,1,2,3,4,5,correlation
would,0.365950,0.341954,0.297668,0.265512,0.194725,-1.0
hour,0.122386,0.094500,0.067052,0.055794,0.042429,-1.0
left,0.120413,0.102786,0.064052,0.043592,0.035293,-1.0
rude,0.117131,0.059593,0.024675,0.010341,0.003916,-1.0
worst,0.114599,0.046093,0.022053,0.007239,0.002537,-1.0
use,0.110522,0.102855,0.099078,0.083270,0.066230,-1.0
wait,0.106092,0.098056,0.069890,0.060869,0.060508,-1.0
phone,0.105150,0.067465,0.037485,0.025980,0.023728,-1.0
line,0.102515,0.097607,0.076890,0.059326,0.033463,-1.0
booked,0.101161,0.082070,0.052134,0.044769,0.039831,-1.0


In [131]:
w[(w['correlation']>0.95) & (w['5']>0.03)].sort_values(by=['correlation'], ascending = False)

,1,2,3,4,5,correlation
Vegas,0.148084,0.182025,0.204913,0.236970,0.242912,1.0
amazing,0.011981,0.026448,0.043161,0.074679,0.154626,1.0
views,0.003061,0.009806,0.017135,0.027635,0.032609,1.0
fantastic,0.003282,0.010462,0.016837,0.029369,0.046589,1.0
guide,0.004195,0.005939,0.006811,0.009689,0.030950,1.0
loved,0.005740,0.014432,0.026161,0.057751,0.064534,1.0
excellent,0.005932,0.017436,0.026378,0.047203,0.062851,1.0
accommodating,0.005946,0.009668,0.013594,0.021700,0.033280,1.0
favorite,0.006049,0.014708,0.025189,0.046630,0.062851,1.0
perfect,0.006697,0.012361,0.024189,0.052135,0.069926,1.0
